In [1]:
import pandas as pd
import numpy as np
import time
import copy

In [2]:
def crossover(population_list):
    
    population_size = len(population_list)

    #初始化母代和後代，選出要拿來交配的母代
    parent_list=copy.deepcopy(population_list)
    offspring_list=copy.deepcopy(population_list)
    S=list(np.random.permutation(population_size))
    
    #選出至多一半人口來交配
    for m in range(int(population_size/2)):
        crossover_prob=np.random.rand()
        if crossover_rate>=crossover_prob:
            parent_1= population_list[S[2*m]][:]
            parent_2= population_list[S[2*m+1]][:]
            child_1=['na' for i in range(20)]
            child_2=['na' for i in range(20)]
            fix_num=round(20/2)
            g_fix=list(np.random.choice(20, fix_num, replace=False))
            
            for g in range(fix_num):
                child_1[g_fix[g]]=parent_2[g_fix[g]]
                child_2[g_fix[g]]=parent_1[g_fix[g]]
            c1=[parent_1[i] for i in range(20) if parent_1[i] not in child_1]
            c2=[parent_2[i] for i in range(20) if parent_2[i] not in child_2]
            
            for i in range(20-fix_num):
                child_1[child_1.index('na')]=c1[i]
                child_2[child_2.index('na')]=c2[i]
            offspring_list[S[2*m]]=child_1[:]
            offspring_list[S[2*m+1]]=child_2[:]
            
    return parent_list, offspring_list

In [3]:
#根據突變率突變後代
def mutation(offspring_list, num_mutation_jobs):
    for m in range(len(offspring_list)):
        mutation_prob=np.random.rand()
        if mutation_rate >= mutation_prob:
            m_chg=list(np.random.choice(20, num_mutation_jobs, replace=False))
            t_value_last=offspring_list[m][m_chg[0]]
            for i in range(num_mutation_jobs-1):
                offspring_list[m][m_chg[i]]=offspring_list[m][m_chg[i+1]] 
            
            offspring_list[m][m_chg[num_mutation_jobs-1]]=t_value_last
    return offspring_list

In [4]:
#calculate total weighted tardiness of job_list
def weightt(jobs_list):
    wt = 0
    for i in range(20):
        sump = 0
        for j in range(i+1):
            sump += jobs_list[j][1]
        w = jobs_list[i][3]
        d = jobs_list[i][2]
        t = max(sump-d,0)
        wt += w*t
    return wt

In [5]:
#calculate tardiness of 母代+後代，同時計算chrom_fitness(取倒數)為selection做準備
def tardiness(jobs, parent_list, offspring_list):
    total_chromosome=copy.deepcopy(parent_list)+copy.deepcopy(offspring_list)
    chrom_fitness,chrom_fit=[],[]
    total_fitness=0
    for i in range(len(total_chromosome)):
        jobs_list= []
        for j in range(20):
            jobs_list.append(jobs[total_chromosome[i][j]-1])
        wt =  weightt(jobs_list)
        chrom_fitness.append(1/wt)
        chrom_fit.append(wt)
        total_fitness=total_fitness+chrom_fitness[i]
    return total_chromosome, chrom_fitness,chrom_fit, total_fitness

In [6]:
def selection(population_size, chrom_fitness, total_fitness):
    pk,qk=[],[]
    
    for i in range(population_size*2):
        pk.append(chrom_fitness[i]/total_fitness)
    for i in range(population_size*2):
        cumulative=0
        for j in range(0,i+1):
            cumulative=cumulative+pk[j]
        qk.append(cumulative)
    
    selection_rand=[np.random.rand() for i in range(population_size)]
    
    for i in range(population_size):
        if selection_rand[i]<=qk[0]:
            population_list[i]=copy.deepcopy(total_chromosome[0])
        else:
            for j in range(0,population_size*2-1):
                if selection_rand[i]>qk[j] and selection_rand[i]<=qk[j+1]:
                    population_list[i]=copy.deepcopy(total_chromosome[j+1])
                    break
    return population_list

In [7]:
#count tardy jobs in job_list
def tardyjobs(jobs_list):
    tj = 0
    for i in range(20):
        sump = 0
        for j in range(i+1):
            sump += jobs_list[j][1]
        d = jobs_list[i][2]
        
        if sump-d > 0:
            tj += 1
    return tj

In [11]:
def GA(population_size, crossover_rate, mutation_rate, mutation_selection_rate, num_mutation_jobs, num_iteration):
    
    jobs =[(1,10, 50, 10), (2,10, 38, 5), (3, 13, 49, 1), (4, 4, 12, 5), (5, 9, 20, 10), (6, 4, 105, 1), (7, 8, 73, 5), (8, 15, 45, 10), 
                (9, 7, 6, 5), (10, 1, 64, 1), (11, 9, 15, 5), (12, 3, 6, 10), (13, 15, 92, 10), (14, 9, 43, 5), (15,11, 78, 1), 
                (16, 6, 21, 10), (17, 5, 15, 5), (18, 14, 50, 5), 
                (19, 18, 150, 1), (20, 3, 99, 5)]

    start_time = time.time()

    wt_best=999999999999999
    best_list,best_obj=[],[]
    population_list=[]

    #產生population_size個jobs seq組合
    for i in range(population_size):
        random_num=list(np.random.permutation(20)) 
        population_list.append(random_num) 

    #
    for n in range(num_iteration):
        wt_best_now=99999999999           
        #定義母、後代
        parent_list, offspring_list = crossover(population_list)

        #後代突變
        offspring_list = mutation(offspring_list, num_mutation_jobs)

        #calculate weighted tardiness
        total_chromosome, chrom_fitness,chrom_fit, total_fitness = tardiness(jobs, parent_list, offspring_list)

        #選出下一輪的population list
        population_list = selection(population_size, chrom_fitness, total_fitness)

        #choose the best solutnion (lowest weighte tardiness) in this iteration
        for i in range(population_size*2):
            if chrom_fit[i]<wt_best_now:
                wt_best_now=chrom_fit[i]
                sequence_now=copy.deepcopy(total_chromosome[i])

        if wt_best_now<=wt_best:
            wt_best=wt_best_now
            sequence_best=copy.deepcopy(sequence_now)

    jobs_list= []
    for j in range(20):
        jobs_list.append(jobs[sequence_best[j]-1])
    num_tardy = tardyjobs(jobs_list)

    return [population_size, crossover_rate, mutation_rate, mutation_selection_rate, \
            num_iteration, wt_best, sequence_best, num_tardy, (time.time() - start_time)]

In [34]:
res = []
psize = [10,30]
cr = [0.2, 0.8]
mr = [0.1, 0.2]
iter = [500,1000]
for p in psize:
    for c in cr:
        for m in mr:
            for i in iter:
                r = GA(p, c, m, 0.5, 10, i)
                res.append(r)

In [35]:
df = pd.DataFrame(res, columns = ['population_size', 'crossover_rate', 'mutation_rate', 'mutation_selection_rate', 'num_iteration',\
                                          'min weighted tardiness', 'optimal job seq.', '# of tardy jobs', 'execution time'])
df.style.set_properties(subset=['optimal job seq.'], **{'width': '400px'})

,population_size,crossover_rate,mutation_rate,mutation_selection_rate,num_mutation_jobs,num_iteration,min weighted tardiness,optimal job seq.,# of tardy jobs,execution time
0,10,0.200000,0.100000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,16.291024
1,10,0.200000,0.100000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,30.961887
2,10,0.200000,0.200000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,17.548960
3,10,0.200000,0.200000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,37.190466
4,10,0.800000,0.100000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,17.877327
5,10,0.800000,0.100000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,34.714847
6,10,0.800000,0.200000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,18.177294
7,10,0.800000,0.200000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,36.106678
8,30,0.200000,0.100000,0.500000,10,500,2200,"[12, 16, 5, 4, 17, 8, 1, 7, 2, 11, 9, 0, 14, 13, 18, 3, 6, 10, 19, 15]",14,17.048352
9,30,0.200000,0.100000,0.500000,10,1000,2282,"[12, 16, 5, 11, 17, 8, 0, 1, 7, 10, 4, 9, 13, 14, 2, 18, 6, 3, 19, 15]",15,35.378904


,population_size,crossover_rate,mutation_rate,mutation_selection_rate,num_mutation_jobs,num_iteration,min weighted tardiness,optimal job seq.,# of tardy jobs,execution time
0,10,0.200000,0.100000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,15.681677
1,10,0.200000,0.100000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,30.359226
2,10,0.200000,0.200000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,14.956187
3,10,0.200000,0.200000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,30.707170
4,10,0.800000,0.100000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,15.676862
5,10,0.800000,0.100000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,31.395488
6,10,0.800000,0.200000,0.500000,10,500,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,15.738492
7,10,0.800000,0.200000,0.500000,10,1000,2392,"[12, 16, 5, 11, 8, 17, 0, 1, 10, 7, 4, 9, 13, 14, 2, 3, 18, 6, 19, 15]",13,30.181054
8,30,0.200000,0.100000,0.500000,10,500,2297,"[9, 12, 16, 11, 5, 17, 8, 10, 1, 7, 4, 13, 14, 0, 2, 3, 18, 6, 19, 15]",16,15.415370
9,30,0.200000,0.100000,0.500000,10,1000,2268,"[9, 12, 16, 5, 11, 17, 8, 1, 10, 14, 4, 7, 13, 0, 2, 3, 18, 6, 19, 15]",19,31.588923
